## Quote报价用法

### 订阅TICK

* 报价采用订阅-推送模式，订阅标的可以是全市场任何标的 (股票/指数/可转债/ETF等都可以)
* 目前限制为：每IP只允许一个连接，每个连接只允许最多订阅`20只`标的
* 数据为实时推送，推送时间为交易日早`9:15:00`-`15:00:00`, 交易时间为3秒推送时间切片, 中午休市为1分钟推送
* 订阅必须是数组，代码必须是9位，沪市以`.SH`结尾, 深市以`.SZ`结尾
* `time`为tick从交易所出来的时间戳，可自行测试时间延迟, 可转债延迟在几十到几百毫秒，股票延迟在几百毫秒到1-2秒都算正常
* 报价为标准的Level1报价，具体说明见下
  
#### Level1数据格式返回示例
```js
{'code': '113616.SH', 'date': 20220913, 'time': 133018802, 'pre_close': 1168330, 'price': 1170210, 'open': 1168340, 'high': 1179500, 'low': 1168340, 'vol': 24425, 'amount': 28604918, 'ask_p1': 1170210, 'ask_v1': 4, 'ask_p2': 1170220, 'ask_v2': 2, 'ask_p3': 1170230, 'ask_v3': 8, 'ask_p4': 1170240, 'ask_v4': 48, 'ask_p5': 1170250, 'ask_v5': 97, 'bid_p1': 1170010, 'bid_v1': 2, 'bid_p2': 1170000, 'bid_v2': 493, 'bid_p3': 1169800, 'bid_v3': 2, 'bid_p4': 1169710, 'bid_v4': 8, 'bid_p5': 1169610, 'bid_v5': 2}
```
几个字段说明:
- 所有价格都需要`除以10000`为实际价格
- `pre_close` 为昨日收盘价
- `ask_p1`-`ask_p5`为`买一`-`买五`, 以此类推

In [1]:
from leekquant.quote import Quote

# 初始化
q = Quote()

# 定义一个回调, 所有收到的tick都在这里
def tick_handler(tick):
    print(tick)

# 注册回调
q.reg_tick_handler(tick_handler)

# 订阅你感兴趣的标的, 有任何tick数据，都会推送到回调函数里
# 最多订阅20只，订阅多了不会报错，也不会推送任何数据
q.sub_tick(codes=['603690.SH','002371.SZ','300812.SZ','300604.SZ','003043.SZ'])

2023-01-09 00:51:08.438 | DEBUG    | leekquant.quote:connect:37 - connection established
2023-01-09 00:51:22.937 | DEBUG    | leekquant.quote:sub_tick:22 - 订阅TICK:


{"code":"603690.SH","name":"至纯科技","date":20220813,"time":101250,"pre_close":428800,"price":441100,"open":431600,"high":444400,"low":425000,"vol":8699238,"amount":380858816,"pct_chg":2.87,"high_limit":471680,"ask_p1":441100,"ask_v1":492530000,"ask_p2":441200,"ask_v2":613000000,"ask_p3":441400,"ask_v3":16000000,"ask_p4":441500,"ask_v4":88000000,"ask_p5":441600,"ask_v5":66000000,"bid_p1":440600,"bid_v1":6000000,"bid_p2":440500,"bid_v2":2000000,"bid_p3":440400,"bid_v3":9000000,"bid_p4":440300,"bid_v4":1000000,"bid_p5":440200,"bid_v5":300000000}
{"code":"002371.SZ","name":"北方华创","date":20220813,"time":101251,"pre_close":2886000,"price":3045400,"open":2886000,"high":3055000,"low":2863300,"vol":5772055,"amount":1726781568,"pct_chg":5.52,"high_limit":3174600,"ask_p1":3045400,"ask_v1":3000000,"ask_p2":3045900,"ask_v2":2000000,"ask_p3":3049800,"ask_v3":7000000,"ask_p4":3049900,"ask_v4":2000000,"ask_p5":3050000,"ask_v5":64000000,"bid_p1":3044900,"bid_v1":9000000,"bid_p2":3043100,"bid_v2":1000000,

### 订阅市场概况
市场概况会主动推送如下数据：
- 沪市大盘
- 深市大盘
- 创业板
- 总的上涨家数 / 总的下跌家数
- 总的涨停家数 / 总的跌停家数
- 炸板率 (之前涨停过又打开涨停 / 所有涨停过的票)
- 北向资金流入 (总额 / 上证流入 / 深证流入)

In [ ]:
def overview_handler(data):
    print('收到市场概况:', data)
q.reg_overview_handler(overview_handler)
q.sub_overview()

#### 返回数据示例
```JSON
{"code":"000001.SH,"date":20230109,"time":112506000,"pre_close":113677321,"price":114350117,"open":114113703,"high":114905947,"low":114086652,"vol":22744801189,"amount":294701143453,"pct_chg":0.59,"wz":["11:25","265472.19","154486.05","419958.24"],"zdt":[35,7,21,62.5],"rise_down":[0,0,0]}
```

#### 返回字段说明
- `code` 可能为: 000001.SH, 399001.SZ, 399006.SZ, 对应 沪市，深市，创业板
- 所有价格需要除以10000
- `pct_chg` 为涨跌幅
- `wz`为北向外资流入情况, 数组4位，分别为: `时间`，`总额`，`沪市流入`，`深市流入`
- `zdt` 为市场张跌停概况，数组4位，分别为: `涨停数`, `炸板数`, `跌停数`, `炸板率`
- `rise_down` 为市场涨跌概况，数组3位，分别为：`上涨总家数`，`平盘总家数`，`下跌总家数`

### 订阅同花顺热点

In [ ]:
## 稍后提供

### 订阅热点对应成分股

In [ ]:
## 稍后提供